# ML with TensorFlow Extended (TFX) -- Part 3
The puprpose of this tutorial is to show how to do end-to-end ML with TFX libraries on Google Cloud Platform. This tutorial covers:
1. Data analysis and schema generation with **TF Data Validation**.
2. Data preprocessing with **TF Transform**.
3. Model training with **TF Estimator**.
4. Model evaluation with **TF Model Analysis**.

This notebook has been tested in Jupyter on the Deep Learning VM.

## 0. Setup Python and Cloud environment

Install libraries we need.

In [ ]:
%pip install -q --upgrade tensorflow_data_validation tensorflow_model_analysis

Note: you may need to restart the kernel to use updated packages.


In [1]:
import apache_beam as beam
import platform
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tornado

print('tornado version: {}'.format(tornado.version))
print('Python version: {}'.format(platform.python_version()))
print('TF version: {}'.format(tf.__version__))
print('TFT version: {}'.format(tft.__version__))
print('TFDV version: {}'.format(tfdv.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


tornado version: 6.0.2
Python version: 3.5.3
TF version: 1.13.1
TFT version: 0.13.0
TFDV version: 0.13.1
Apache Beam version: 2.11.0


In [2]:
PROJECT = 'cloud-training-demos'    # Replace with your PROJECT
BUCKET = 'cloud-training-demos-ml'  # Replace with your BUCKET
REGION = 'us-central1'              # Choose an available region for Cloud MLE

import os

os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## ensure we predict locally with our current Python environment
gcloud config set ml_engine/local_python `which python`

Updated property [core/project].
Updated property [compute/region].
Updated property [ml_engine/local_python].


<img valign="middle" src="images/tfx.jpeg">

### UCI Adult Dataset: https://archive.ics.uci.edu/ml/datasets/adult
Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset.

In [4]:
DATA_DIR='gs://cloud-samples-data/ml-engine/census/data'

In [5]:
import os

TRAIN_DATA_FILE = os.path.join(DATA_DIR, 'adult.data.csv')
EVAL_DATA_FILE = os.path.join(DATA_DIR, 'adult.test.csv')
!gsutil ls -l $TRAIN_DATA_FILE
!gsutil ls -l $EVAL_DATA_FILE

   3974304  2018-05-10T22:21:14Z  gs://cloud-samples-data/ml-engine/census/data/adult.data.csv
TOTAL: 1 objects, 3974304 bytes (3.79 MiB)
   1986465  2018-05-10T22:21:14Z  gs://cloud-samples-data/ml-engine/census/data/adult.test.csv
TOTAL: 1 objects, 1986465 bytes (1.89 MiB)


In [6]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
               'marital_status', 'occupation', 'relationship', 'race', 'gender',
               'capital_gain', 'capital_loss', 'hours_per_week',
               'native_country', 'income_bracket']

TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'

RAW_SCHEMA_LOCATION = 'raw_schema.pbtxt'

## 3. Model Training
For training the model, we use [TF Estimators](https://www.tensorflow.org/guide/estimators) APIs to train a premade DNNClassifier. We perform the following:
1. Load the **transform schema**
2. Use the transform schema to parse TFRecords in **input_fn**
3. Use the transform schema to create **feature columns**
4. Create a premade **DNNClassifier**
5. **Train** the model
6. Implement the **serving_input_fn** and apply the **transform logic**
7. **Export** and test the saved model.

### 3.1 Load transform output

In [7]:
PREPROC_OUTPUT_DIR = 'gs://{}/census/tfx'.format(BUCKET)  # from 02_transform.ipynb
TRANSFORM_ARTIFACTS_DIR = os.path.join(PREPROC_OUTPUT_DIR,'transform')
TRANSFORMED_DATA_DIR = os.path.join(PREPROC_OUTPUT_DIR,'transformed')
!gsutil ls $TRANSFORM_ARTIFACTS_DIR
!gsutil ls $TRANSFORMED_DATA_DIR

gs://cloud-training-demos-ml/census/tfx/transform/
gs://cloud-training-demos-ml/census/tfx/transform/transform_fn/
gs://cloud-training-demos-ml/census/tfx/transform/transformed_metadata/
gs://cloud-training-demos-ml/census/tfx/transformed/eval-00000-of-00003.tfrecords
gs://cloud-training-demos-ml/census/tfx/transformed/eval-00001-of-00003.tfrecords
gs://cloud-training-demos-ml/census/tfx/transformed/eval-00002-of-00003.tfrecords
gs://cloud-training-demos-ml/census/tfx/transformed/train-00000-of-00004.tfrecords
gs://cloud-training-demos-ml/census/tfx/transformed/train-00001-of-00004.tfrecords
gs://cloud-training-demos-ml/census/tfx/transformed/train-00002-of-00004.tfrecords
gs://cloud-training-demos-ml/census/tfx/transformed/train-00003-of-00004.tfrecords


In [8]:
transform_output = tft.TFTransformOutput(TRANSFORM_ARTIFACTS_DIR)

### 3.2 TFRecords Input Function

In [9]:
def make_input_fn(tfrecords_files, 
  batch_size, num_epochs=1, shuffle=False):

  def input_fn():
    dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=tfrecords_files,
      batch_size=batch_size,
      features=transform_output.transformed_feature_spec(),
      label_key=TARGET_FEATURE_NAME,
      reader=tf.data.TFRecordDataset,
      num_epochs=num_epochs,
      shuffle=shuffle
    )
    return dataset

  return input_fn

In [10]:
make_input_fn(TRANSFORMED_DATA_DIR+'/train*.tfrecords', 2, shuffle=False)()

<DatasetV1Adapter shapes: ({education_num_scaled: (?, 1), native_country_integerized: (?, 1), capital_gain_scaled: (?, 1), relationship_integerized: (?, 1), hours_per_week_scaled: (?, 1), marital_status_integerized: (?, 1), age_scaled: (?, 1), capital_loss_scaled: (?, 1), gender_integerized: (?, 1), race_integerized: (?, 1), education_integerized: (?, 1), age_bucketized: (?, 1), workclass_integerized: (?, 1), occupation_integerized: (?, 1)}, (?, 1)), types: ({education_num_scaled: tf.float32, native_country_integerized: tf.int64, capital_gain_scaled: tf.float32, relationship_integerized: tf.int64, hours_per_week_scaled: tf.float32, marital_status_integerized: tf.int64, age_scaled: tf.float32, capital_loss_scaled: tf.float32, gender_integerized: tf.int64, race_integerized: tf.int64, education_integerized: tf.int64, age_bucketized: tf.int64, workclass_integerized: tf.int64, occupation_integerized: tf.int64}, tf.string)>

### 3.3 Create feature columns

In [11]:
import math

def create_feature_columns():

  feature_columns = []
  transformed_features = transform_output.transformed_metadata.schema._schema_proto.feature

  for feature in transformed_features:

    if feature.name in [TARGET_FEATURE_NAME, WEIGHT_COLUMN_NAME]:
      continue

    if hasattr(feature, 'int_domain') and feature.int_domain.is_categorical:
      vocab_size = feature.int_domain.max + 1
      feature_columns.append(
        tf.feature_column.embedding_column(
          tf.feature_column.categorical_column_with_identity(
            feature.name, num_buckets=vocab_size),
            dimension = int(math.sqrt(vocab_size))))
    else:
      feature_columns.append(
        tf.feature_column.numeric_column(feature.name))

  return feature_columns

In [12]:
create_feature_columns()

[EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='age_bucketized', number_buckets=5, default_value=None), dimension=2, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fa8db5b96a0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 NumericColumn(key='age_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='capital_gain_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='capital_loss_scaled', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='education_integerized', number_buckets=16, default_value=None), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fa89f1807f0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 NumericColumn(key

### 3.4 Instantiate and Estimator

In [13]:
def create_estimator(params, run_config):
    
  feature_columns = create_feature_columns()

  estimator = tf.estimator.DNNClassifier(
    #weight_column=WEIGHT_COLUMN_NAME,
    label_vocabulary=TARGET_LABELS,
    feature_columns=feature_columns,
    hidden_units=params.hidden_units,
    config=run_config
  )

  return estimator

### 3.5 Implement train and evaluate experiment

In [14]:
from datetime import datetime

def run_experiment(estimator, params, run_config, resume=False):
  
  tf.logging.set_verbosity(tf.logging.INFO)

  if not resume: 
    if tf.gfile.Exists(run_config.model_dir):
      print("Removing previous artifacts...")
      tf.gfile.DeleteRecursively(run_config.model_dir)
  else:
    print("Resuming training...")

  train_spec = tf.estimator.TrainSpec(
      input_fn = make_input_fn(
          TRANSFORMED_DATA_DIR+'/train*.tfrecords',
          batch_size=params.batch_size,
          num_epochs=None,
          shuffle=True
      ),
      max_steps=params.max_steps
  )

  eval_spec = tf.estimator.EvalSpec(
      input_fn = make_input_fn(
          TRANSFORMED_DATA_DIR+'/eval*.tfrecords',
          batch_size=params.batch_size,     
      ),
      start_delay_secs=0,
      throttle_secs=0,
      steps=None
  )
  
  time_start = datetime.utcnow() 
  print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
  print(".......................................")
  
  tf.estimator.train_and_evaluate(
    estimator=estimator,
    train_spec=train_spec, 
    eval_spec=eval_spec)

  time_end = datetime.utcnow() 
  print(".......................................")
  print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
  print("")
  
  time_elapsed = time_end - time_start
  print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))

### 3.5 Run experiment

In [15]:
MODELS_LOCATION = 'models/census'
MODEL_NAME = 'dnn_classifier'
model_dir = os.path.join(MODELS_LOCATION, MODEL_NAME)
os.environ['MODEL_DIR'] = model_dir

params = tf.contrib.training.HParams()
params.hidden_units = [128, 64]
params.dropout = 0.15
params.batch_size =  128
params.max_steps = 1000

run_config = tf.estimator.RunConfig(
    tf_random_seed=19831006,
    save_checkpoints_steps=200, 
    keep_checkpoint_max=3, 
    model_dir=model_dir,
    log_step_count_steps=10
)

In [16]:
estimator = create_estimator(params, run_config)
run_experiment(estimator, params, run_config)

INFO:tensorflow:Using config: {'_service': None, '_model_dir': 'models/census/dnn_classifier', '_protocol': None, '_keep_checkpoint_max': 3, '_evaluation_master': '', '_train_distribute': None, '_global_id_in_cluster': 0, '_device_fn': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_task_id': 0, '_num_worker_replicas': 1, '_log_step_count_steps': 10, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_eval_distribute': None, '_save_checkpoints_secs': None, '_experimental_distribute': None, '_save_checkpoints_steps': 200, '_master': '', '_tf_random_seed': 19831006, '_save_summary_steps': 100, '_task_type': 'worker', '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa89f189128>}
Removing previous artifacts...
Experiment started at 22:43:36
.......................................
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorf

### 3.6 Export the model for serving

In [17]:
tf.logging.set_verbosity(tf.logging.ERROR)

def make_serving_input_receiver_fn():
  from tensorflow_transform.tf_metadata import schema_utils

  source_raw_schema = tfdv.load_schema_text(RAW_SCHEMA_LOCATION)
  raw_feature_spec = schema_utils.schema_as_feature_spec(source_raw_schema).feature_spec
  raw_feature_spec.pop(TARGET_FEATURE_NAME)
  if WEIGHT_COLUMN_NAME in raw_feature_spec:
    raw_feature_spec.pop(WEIGHT_COLUMN_NAME)


  # Create the interface for the serving function with the raw features
  raw_features = tf.estimator.export.build_parsing_serving_input_receiver_fn(raw_feature_spec)().features

  receiver_tensors = {feature: tf.placeholder(shape=[None], dtype=raw_features[feature].dtype) 
    for feature in raw_features
  }

  receiver_tensors_expanded = {tensor: tf.reshape(receiver_tensors[tensor], (-1, 1)) 
    for tensor in receiver_tensors
  }

  # Apply the transform function 
  transformed_features = transform_output.transform_raw_features(receiver_tensors_expanded)

  return tf.estimator.export.ServingInputReceiver(
    transformed_features, receiver_tensors)

In [20]:
export_dir = os.path.join(model_dir, 'export')

if tf.gfile.Exists(export_dir):
    tf.gfile.DeleteRecursively(export_dir)
        
estimator.export_savedmodel(
    export_dir_base=export_dir,
    serving_input_receiver_fn=make_serving_input_receiver_fn
)

b'models/census/dnn_classifier/export/1554072293'

In [21]:
%%bash

saved_models_base=${MODEL_DIR}/export/
saved_model_dir=${MODEL_DIR}/export/$(ls ${saved_models_base} | tail -n 1)
echo ${saved_model_dir}
saved_model_cli show --dir=${saved_model_dir} --all

models/census/dnn_classifier/export/1554072293

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['age'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_6:0
    inputs['capital_gain'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_8:0
    inputs['capital_loss'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_3:0
    inputs['education'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: Placeholder:0
    inputs['education_num'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_4:0
    inputs['gender'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: Placeholder_7:0
    inputs['hours_per_week'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_1:0


### 3.7 Try out saved model

In [22]:
export_dir = os.path.join(model_dir, 'export')
tf.gfile.ListDirectory(export_dir)[-1]
saved_model_dir = os.path.join(export_dir, tf.gfile.ListDirectory(export_dir)[-1])
print(saved_model_dir)
print()

predictor_fn = tf.contrib.predictor.from_saved_model(
    export_dir = saved_model_dir,
    signature_def_key="predict"
)

input = {
        'age': [34.0],
        'workclass': ['Private'],
        'education': ['Doctorate'],
        'education_num': [10.0],
        'marital_status': ['Married-civ-spouse'],
        'occupation': ['Prof-specialty'],
        'relationship': ['Husband'],
        'race': ['White'],
        'gender': ['Male'],
        'capital_gain': [0.0], 
        'capital_loss': [0.0], 
        'hours_per_week': [40.0],
        'native_country':['Mexico']
}

print(input)
print()
output = predictor_fn(input)
print(output)

models/census/dnn_classifier/export/1554072293

{'education': ['Doctorate'], 'hours_per_week': [40.0], 'race': ['White'], 'capital_loss': [0.0], 'education_num': [10.0], 'workclass': ['Private'], 'native_country': ['Mexico'], 'age': [34.0], 'gender': ['Male'], 'marital_status': ['Married-civ-spouse'], 'occupation': ['Prof-specialty'], 'capital_gain': [0.0], 'relationship': ['Husband']}

{'logistic': array([[0.2034915]], dtype=float32), 'probabilities': array([[0.79650855, 0.20349152]], dtype=float32), 'class_ids': array([[0]]), 'classes': array([[b' <=50K']], dtype=object), 'logits': array([[-1.3646135]], dtype=float32)}


### 3.8 Deploy model to Cloud ML Engine

In [23]:
#%%bash
#MODEL_NAME="census"
#MODEL_VERSION="v1"
#MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/census/dnn_classifier/export/exporter | tail -1)
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
#gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version 1.13

### 3.9 Export evaluation saved model

In [31]:
HEADER_DEFAULTS = [[0], [''], [0], [''], [0], [''], [''], [''], [''], [''],
                   [0], [0], [0], [''], ['']]

def make_eval_input_receiver_fn():
  receiver_tensors = {'examples': tf.placeholder(dtype=tf.string, shape=[None])}
  columns = tf.decode_csv(receiver_tensors['examples'], record_defaults=HEADER_DEFAULTS)
  features = dict(zip(HEADER, columns))
  print(features)

  for feature_name in features:
    if features[feature_name].dtype == tf.int32:
      features[feature_name] = tf.cast(features[feature_name], tf.int64)
    features[feature_name] = tf.reshape(features[feature_name], (-1, 1))

  transformed_features = transform_output.transform_raw_features(features)
  features.update(transformed_features)

  return tfma.export.EvalInputReceiver(
    features=features,
    receiver_tensors=receiver_tensors,
    labels=features[TARGET_FEATURE_NAME]
    )

In [32]:
import tensorflow_model_analysis as tfma
eval_model_dir = os.path.join(model_dir, "export/evaluate")
if tf.gfile.Exists(eval_model_dir):
    tf.gfile.DeleteRecursively(eval_model_dir)

tfma.export.export_eval_savedmodel(
        estimator=estimator,
        export_dir_base=eval_model_dir,
        eval_input_receiver_fn=make_eval_input_receiver_fn
)

{'education': <tf.Tensor 'DecodeCSV:3' shape=(?,) dtype=string>, 'hours_per_week': <tf.Tensor 'DecodeCSV:12' shape=(?,) dtype=int32>, 'race': <tf.Tensor 'DecodeCSV:8' shape=(?,) dtype=string>, 'capital_loss': <tf.Tensor 'DecodeCSV:11' shape=(?,) dtype=int32>, 'education_num': <tf.Tensor 'DecodeCSV:4' shape=(?,) dtype=int32>, 'workclass': <tf.Tensor 'DecodeCSV:1' shape=(?,) dtype=string>, 'native_country': <tf.Tensor 'DecodeCSV:13' shape=(?,) dtype=string>, 'age': <tf.Tensor 'DecodeCSV:0' shape=(?,) dtype=int32>, 'gender': <tf.Tensor 'DecodeCSV:9' shape=(?,) dtype=string>, 'marital_status': <tf.Tensor 'DecodeCSV:5' shape=(?,) dtype=string>, 'income_bracket': <tf.Tensor 'DecodeCSV:14' shape=(?,) dtype=string>, 'occupation': <tf.Tensor 'DecodeCSV:6' shape=(?,) dtype=string>, 'capital_gain': <tf.Tensor 'DecodeCSV:10' shape=(?,) dtype=int32>, 'relationship': <tf.Tensor 'DecodeCSV:7' shape=(?,) dtype=string>, 'fnlwgt': <tf.Tensor 'DecodeCSV:2' shape=(?,) dtype=int32>}


ValueError: Unexpected inputs to transform: {'fnlwgt'}

## License

Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

---
This is not an official Google product. The sample code provided for educational purposes only.
---